# ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

<img src="https://github.com/GoogleCloudPlatform/agent-starter-pack/blob/main/docs/images/adk_logo.png?raw=true" width="400">


## Install dependencies

In [1]:
!pip install google-cloud-aiplatform --upgrade

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

### Import libraries

In [7]:
import json

import google.auth
import requests
import vertexai
import vertexai.agent_engines
from IPython.display import Markdown, display


### Ensure We're Authenticating to the Right Project

In [8]:
credentials, project_id = google.auth.default()
vertexai.init(project="rickbot-adk-dev", location="europe-west4", credentials=credentials)

## If you are using Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [9]:
# Replace with your Agent Engine ID
# AGENT_ENGINE_ID = "projects/PROJECT_ID/locations/us-central1/reasoningEngines/ENGINE_ID"
AGENT_ENGINE_ID = "projects/rickbot-adk-dev/locations/europe-west4/reasoningEngines/1853090509174603776"

In [10]:
remote_agent_engine = vertexai.agent_engines.get(AGENT_ENGINE_ID)

In [11]:
for event in remote_agent_engine.stream_query(message="hi!", user_id="test"):
    display(event)

{'content': {'parts': [{'text': 'Hello! How can I help you today?\n'}],
  'role': 'model'},
 'usage_metadata': {'candidates_token_count': 10,
  'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 10}],
  'prompt_token_count': 147,
  'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 147}],
  'total_token_count': 157,
  'traffic_type': 'ON_DEMAND'},
 'invocation_id': 'e-5237a2f3-8bc2-42e1-a7ea-d3f2684ffe63',
 'author': 'root_agent',
 'actions': {'state_delta': {},
  'artifact_delta': {},
  'requested_auth_configs': {}},
 'id': '51989ab6-af39-420a-af94-5ca74cc8bf7b',
 'timestamp': 1754036601.425867}

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [2]:
# Uncomment the following lines if you're not using the virtual environment created by uv
# import sys
# sys.path.append("../")

In [13]:
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [14]:
for event in agent_engine.stream_query(message="hi!", user_id="test"):
    display(event)

{'content': {'parts': [{'text': 'Hi there! How can I help you today?'}],
  'role': 'model'},
 'usage_metadata': {'candidates_token_count': 10,
  'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
    'token_count': 10}],
  'prompt_token_count': 147,
  'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
    'token_count': 147}],
  'thoughts_token_count': 30,
  'total_token_count': 187,
  'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>},
 'invocation_id': 'e-e2d05a9e-a77a-453a-a703-14d75863134a',
 'author': 'root_agent',
 'actions': {'state_delta': {},
  'artifact_delta': {},
  'requested_auth_configs': {}},
 'id': '68429aca-8a95-482e-b264-0a1223066e6c',
 'timestamp': 1754036640.882386}

## If you are using Cloud Run

#### Remote Testing

For more information about authenticating HTTPS requests to Cloud Run services, see:
[Cloud Run Authentication Documentation](https://cloud.google.com/run/docs/triggering/https-request)

Remote testing involves using a deployed service URL instead of localhost.

Authentication is handled using GCP identity tokens instead of local credentials.

In [20]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
SERVICE_URL = "YOUR_SERVICE_URL_HERE"  # Replace with your Cloud Run service URL

You'll need to first create a Session

In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"{SERVICE_URL}/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

Then you will be able to send a message

In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = f"{SERVICE_URL}/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")

### Local Testing

> You can run the application locally via the `make local-backend` command.

#### Create a session
 Create a new session with user preferences and state information


In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"http://127.0.0.1:8000/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

#### Send a message
Send a message to the backend service and receive a streaming response


In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = "http://127.0.0.1:8000/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")